In [42]:
import pandas as pd
import os
import linktransformer as lt
import re


In [43]:
df1 = pd.read_excel('../data/Houston SAP Materials.xlsx', engine='openpyxl')

In [44]:
df1.columns = [re.sub(r'[^a-zA-Z0-9]', "_",\
                         col.replace(' ', "_")).upper() for col in df1.columns]
df1.head()

,MATNR,SHORT_TEXT,BASIC_DATA_TEXT,PURCHASE_ORDER__TEXT,PLANT,SIZE_DIM,MATL,MFR,MFR_NAME,MFR_PART_NO,MRP_TY,LOT_SIZE,SOURCE,ALL_TEXTS
0,1100298596,"""O""RING 1AP22 S/N'S: ST-1843 AND ST-1844",NaN,ORING 1AP22 S/N'S: ST-1843 AND ST-1844 BEGIN ...,4162,NaN,NaN,MCOMML,COMMERCIAL,MHI-131373 OR 155377,ZB,ZB,SAP,"""O""RING 1AP22 S/N'S: ST-1843 AND ST-1844 | ORI..."
1,1100295537,#5 TYGOTHANE TUBE (5 PER PACK) - #5 TYGO,NaN,#5 TYGOTHANE TUBE (5 PER PACK) - #5 TYGOTHANE ...,4162,NaN,NaN,MCOMML,COMMERCIAL,NaN,ZB,ZB,SAP,#5 TYGOTHANE TUBE (5 PER PACK) - #5 TYGO | #5 ...
2,1100293126,"1-1/2"" ALUMINUM UNISTRUT CLAMP BLIN B200",NaN,1-1/2 ALUMINUM UNISTRUT CLAMP BLIN B20099PA-AL...,4162,NaN,NaN,MCOMML,COMMERCIAL,BLIN B20099PA-AL,ZB,ZB,SAP,"1-1/2"" ALUMINUM UNISTRUT CLAMP BLIN B200 | 1-1..."
3,1100298002,"1-1/2 "" BREATHER CAP METAL SCREEN HPGK",NaN,1-1/2 BREATHER CAP METAL SCREEN HPGK,4162,NaN,NaN,MCOMML,COMMERCIAL,NaN,ZB,ZB,SAP,"1-1/2 "" BREATHER CAP METAL SCREEN HPGK | 1-1/2..."
4,1100298001,"1-1/4 "" BREATHER CAP METAL SCREEN HPGJ",NaN,1-1/4 BREATHER CAP METAL SCREEN HPGJ,4162,NaN,NaN,MCOMML,COMMERCIAL,NaN,ZB,ZB,SAP,"1-1/4 "" BREATHER CAP METAL SCREEN HPGJ | 1-1/4..."


In [45]:
df2 = pd.read_excel('../data/Material list 011625.xlsx', engine='openpyxl', skiprows=1)

In [46]:
df2.columns = [re.sub(r'[^a-zA-Z0-9]', "_",\
                            col.replace(' ', "_")).upper() for col in df2.columns]
df2.head()

,PROJECT_NAME,SCOPE_ITEM__,NOTIFICATION_,PROJECT_,MM_,DESCRIPTION,MATERIAL_GROUP,VENDOR_DESCRIPTION,UNIT,QTY,COST_EACH,TOTAL,KIT_
0,2026 HCP Turnaround,1017.0,2140C,A007377,NaN,"5/8"" X 3-1/2"", BOLT: ASTM A193 Grade B7 / AS...",NaN,GHX,EA,16.0,1.44,23.04,2026HCP-1017-C
1,2026 HCP Turnaround,1084.0,109EA,A007377,NaN,GASKET FLEX GRAPHITE 304 SS INNER CS OUTER SP...,NaN,NaN,EA,1.0,0.00,0.00,2026HCP-1084-D
2,2026 HCP Turnaround,1135.0,KV-1706,A007377,NaN,GASKET FLEX GRAPHITE 304SS INNER CS OUTER SPI...,NaN,NaN,EA,3.0,0.00,0.00,2026HCP-1135-B
3,2026 HCP Turnaround,1138.0,KV-1101,A007377,NaN,GASKET FLEX GRAPHITE 304SS INNER CS OUTER SPI...,NaN,NaN,EA,1.0,0.00,0.00,2026HCP-1138-B
4,2026 HCP Turnaround,1133.0,KV-1306,A007377,NaN,GASKET RTJ 310SS OVAL RING MAX BRINELL 170HB ...,NaN,Lamons,EA,2.0,455.46,910.92,2026HCP-1133-B


In [47]:
# Function to clean, tokenize, and convert to lowercase
def clean_and_tokenize(text):
    # Replace all punctuation with spaces
    cleaned_text = re.sub(r'[^\w\s]', ' ', text)
    # Convert to lowercase
    cleaned_text = cleaned_text.lower()
    # # Split into tokens
    # tokens = cleaned_text.split()
    return cleaned_text

In [48]:
df2['DESCRIPTION'].dtypes

dtype('O')

In [49]:
df1['SAP_TEXT'] = df1['PURCHASE_ORDER__TEXT'].apply(clean_and_tokenize)
df2['PLANT_TEXT'] = df2['DESCRIPTION'].astype('str').apply(clean_and_tokenize)

In [54]:
df_lm_matched = lt.merge(df1, df2, merge_type='1:m', model="all-MiniLM-L6-v2", left_on="SAP_TEXT", right_on="PLANT_TEXT")
df_lm_matched.head(10)

2025-01-22 13:12:40 - Use pytorch device_name: mps
2025-01-22 13:12:40 - Load pretrained SentenceTransformer: all-MiniLM-L6-v2


Batches: 100%|██████████| 17/17 [00:00<00:00, 24.87it/s]


,MATNR,SHORT_TEXT,BASIC_DATA_TEXT,PURCHASE_ORDER__TEXT,PLANT,SIZE_DIM,MATL,MFR,MFR_NAME,MFR_PART_NO,...,MATERIAL_GROUP,VENDOR_DESCRIPTION,UNIT,QTY,COST_EACH,TOTAL,KIT_,PLANT_TEXT,id_lt_y,score
0,1100298596,"""O""RING 1AP22 S/N'S: ST-1843 AND ST-1844",NaN,ORING 1AP22 S/N'S: ST-1843 AND ST-1844 BEGIN ...,4162,NaN,NaN,MCOMML,COMMERCIAL,MHI-131373 OR 155377,...,NaN,GHX,EA,33.0,34.55,1140.15,2026HCP-1000-B,stud b 2h ns sa320 l7 sa194 7l u 1 1 4 x 8un...,1834,0.491278
1,1100295537,#5 TYGOTHANE TUBE (5 PER PACK) - #5 TYGO,NaN,#5 TYGOTHANE TUBE (5 PER PACK) - #5 TYGOTHANE ...,4162,NaN,NaN,MCOMML,COMMERCIAL,NaN,...,NaN,NaN,EA,2.0,0.00,0.00,2026HCP-1036-B,gasket 1 pass comp fiber teadit 1001 part ...,1398,0.415222
2,1100293126,"1-1/2"" ALUMINUM UNISTRUT CLAMP BLIN B200",NaN,1-1/2 ALUMINUM UNISTRUT CLAMP BLIN B20099PA-AL...,4162,NaN,NaN,MCOMML,COMMERCIAL,BLIN B20099PA-AL,...,NaN,NaN,EA,24.0,0.00,0.00,6HCP-1030-A,1 1 2 x9 1 2 bolt astm a193 b7 astm a194 2h...,273,0.494838
3,1100298002,"1-1/2 "" BREATHER CAP METAL SCREEN HPGK",NaN,1-1/2 BREATHER CAP METAL SCREEN HPGK,4162,NaN,NaN,MCOMML,COMMERCIAL,NaN,...,NaN,NaN,EA,3.0,0.00,0.00,2026HCP-1205-B,gasket compressed non asbestos klingersil c440...,1403,0.450560
4,1100298001,"1-1/4 "" BREATHER CAP METAL SCREEN HPGJ",NaN,1-1/4 BREATHER CAP METAL SCREEN HPGJ,4162,NaN,NaN,MCOMML,COMMERCIAL,NaN,...,NaN,NaN,EA,1.0,0.00,0.00,2026HCP-1253-B,1 1 2 fitting cs astm a105 plug round he...,244,0.423456
5,1100293125,"1"" ALUMINUM UNISTRUT CLAMP BLIN B2010PAA",NaN,1 ALUMINUM UNISTRUT CLAMP BLIN B2010PAAL 1 AL ...,4162,NaN,NaN,MCOMML,COMMERCIAL,BLIN B2010PAAL,...,NaN,NaN,EA,2.0,0.00,0.00,2026HCP-1176-B,2 fitting cs astm a105 90 degree elbow t...,435,0.431005
6,1100298000,"1"" BREATHER CAP METAL SCREEN HPGA",NaN,1 BREATHER CAP METAL SCREEN HPGA,4162,NaN,NaN,MCOMML,COMMERCIAL,NaN,...,NaN,NaN,EA,1.0,0.00,0.00,2026HCP-1253-B,1 1 2 fitting cs astm a105 plug round he...,244,0.352420
7,1100294278,"1"" CAMLOCK TYPE A MALE ADAPTER X FNPT DI",NaN,1 CAMLOCK TYPE A MALE ADAPTER X FNPT DIXON - P...,4162,NaN,NaN,MCOMML,COMMERCIAL,NaN,...,NaN,NaN,EA,2.0,0.00,0.00,2026HCP-1176-B,2 fitting cs astm a105 90 degree elbow t...,435,0.424951
8,1100294279,"1"" CAMLOCK TYPE D COUPLER DIXON P/N 100-",NaN,1 CAMLOCK TYPE D COUPLER DIXON P/N 100-D-SS,4162,NaN,NaN,MCOMML,COMMERCIAL,NaN,...,NaN,NaN,EA,1.0,0.00,0.00,2026HCP-1072-B,102 o d x 8 thk demister pad vane type t 3...,230,0.478863
9,1100293109,"1"" FLEX CONNECTOR 45 DEGREE TNB5244",NaN,1 FLEX CONNECTOR 45 DEGREE TNB5244,4162,NaN,NaN,MCOMML,COMMERCIAL,TNB5244,...,NaN,NaN,EA,1.0,0.00,0.00,2026HCP-1253-B,1 1 2 fitting cs astm a105 plug round he...,244,0.500083


In [55]:
df_lm_matched.sort_values(by='score', ascending=False, inplace=False)

,MATNR,SHORT_TEXT,BASIC_DATA_TEXT,PURCHASE_ORDER__TEXT,PLANT,SIZE_DIM,MATL,MFR,MFR_NAME,MFR_PART_NO,...,MATERIAL_GROUP,VENDOR_DESCRIPTION,UNIT,QTY,COST_EACH,TOTAL,KIT_,PLANT_TEXT,id_lt_y,score
3777,1100300458,"TUBING FITTING 3/8"" T X 1/4"" MALE NPT CO",NaN,TUBING FITTING 3/8 T X 1/4 MALE NPT CONNECTOR ...,4162,NaN,NaN,MCOMML,COMMERCIAL,ISSD6MC4,...,NaN,NaN,EA,1.0,0.00,0.00,2026HCP-1178-B,tubing fitting 3 4 tube od x 3 4 male npt 3...,2109,0.884560
3775,1100300460,"TUBING FITTING 3/8"" T X 1/2"" MALE NPT CO",NaN,TUBING FITTING 3/8 T X 1/2 MALE NPT CONNECTOR ...,4162,NaN,NaN,MCOMML,COMMERCIAL,ISSD6MC8,...,NaN,NaN,EA,1.0,0.00,0.00,2026HCP-1178-B,tubing fitting 3 4 tube od x 3 4 male npt 3...,2109,0.877210
3779,1100300459,"TUBING FITTING 3/8"" T X 3/8"" MALE NPT CO",NaN,TUBING FITTING 3/8 T X 3/8 MALE NPT CONNECTOR ...,4162,NaN,NaN,MCOMML,COMMERCIAL,ISSD6MC6,...,NaN,NaN,EA,1.0,0.00,0.00,2026HCP-1178-B,tubing fitting 3 4 tube od x 3 4 male npt 3...,2109,0.877009
3765,1100300449,"TUBING FITTING 1/4"" T X 3/8"" MALE NPT CO",NaN,TUBING FITTING 1/4 T X 3/8 MALE NPT CONNECTOR ...,4162,NaN,NaN,MCOMML,COMMERCIAL,ISSD4MC6,...,NaN,NaN,EA,1.0,0.00,0.00,2026HCP-1178-B,tubing fitting 3 4 tube od x 3 4 male npt 3...,2109,0.872228
3756,1100300469,"TUBING FITTING 1/2"" T X 3/8"" MALE NPT CO",NaN,TUBING FITTING 1/2 T X 3/8 MALE NPT CONNECTOR ...,4162,NaN,NaN,MCOMML,COMMERCIAL,ISSD8MC6,...,NaN,NaN,EA,1.0,0.00,0.00,2026HCP-1178-B,tubing fitting 3 4 tube od x 3 4 male npt 3...,2109,0.869434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,1100296704,"CONTROLLER, OVER-SPEED CONTROLLER WOODWA",NaN,"CONTROLLER, OVER-SPEED CONTROLLER WOODWARD PEA...",4162,NaN,NaN,MCOMML,COMMERCIAL,9905-864,...,NaN,NaN,EA,1.0,0.00,0.00,6HCP-1548-B,2 cs astm a216 grade wcb gate os y full po...,529,0.194715
560,1100293383,"BATTERY,DRY CELL, SMART POWER MODULE FOR",NaN,"BATTERY,DRY CELL, SMART POWER MODULE FOR WIREL...",4162,NaN,NaN,MCOMML,COMMERCIAL,EMERSON-701PBKKF,...,NaN,NaN,EA,32.0,0.00,0.00,2026HCP1058-B,anode cathodic protection services type 15s ...,1240,0.194083
454,1100297630,"ANALYZER, COMMUNICATION AND CONTROL BOAR",NaN,"ANALYZER, COMMUNICATION AND CONTROL BOARD, (CA...",4162,NaN,NaN,MCOMML,COMMERCIAL,NaN,...,NaN,NaN,EA,2.0,0.00,0.00,6HCP-1030-A,see 1030 gasket dk2021 024 dtl 002,1818,0.189485
1076,1100296687,"CONTROLLER, POWER SUPPLY",NaN,"CONTROLLER, POWER SUPPLY",4162,NaN,NaN,MCOMML,COMMERCIAL,ALLENBRADLY-1606-XLE240EN,...,NaN,NaN,EA,3.0,0.00,0.00,2026HCP-1012-B,gasket v 6502 t0 12 3 see drawing dhpa r50...,1631,0.189309
